In [1]:
import jieba
import re
import string
import numpy as np
import math
import pandas as pd

In [2]:
# 讀入gept初、中、中高之字典
word1 = open('gept1.txt','r')
word2 = open('gept2.txt','r')
word3 = open('gept3.txt','r')
gept1 = word1.read().split(sep = '\n')
gept2 = word2.read().split(sep = '\n')
gept3 = word3.read().split(sep = '\n')

In [3]:
#讀入文章、分類及停止詞庫
news = open('Content.txt','r').read().split(sep = '^')
catagory = open('Catagory.txt','r').read().split(sep = '\n')
stopword = open('stopwords.txt','r').read().split(sep = '\n')

In [4]:
#刪除中文標點符號
content1 = []
for i in range(0,len(news)):        
    new = re.sub('[！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.]', '', news[i])
    content1.append(new)

#刪除英文標點符號
content = []
for j in range(0,len(content1)):
    n = re.sub('[%s]' % re.escape(string.punctuation), '', content1[j])
    content.append(n)

In [5]:
#定義停止詞及刪除空白
space = ' '
stopword.append(space)
def removeStopword(word_list,stop):
    return [w for w in word_list if w not in stop]

In [6]:
#使用結巴斷詞
content_list = []
for c in range(0,len(content)):
    lst = removeStopword(jieba.lcut(content[c].lower(),cut_all=False),stopword)
    content_list.append(lst)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/qh/8bdz64w16cv0_y5098f1g5440000gn/T/jieba.cache
Loading model cost 1.249 seconds.
Prefix dict has been built succesfully.


In [7]:
#加入字典，並去除停止詞
content_list.append(removeStopword(gept1,stopword))
content_list.append(removeStopword(gept2,stopword))
content_list.append(removeStopword(gept3,stopword))

In [8]:
#得到全文章的term dictionary
term_dict = []
for i in range(0,len(content_list)):
    term_dict.extend(content_list[i])
term_dict = list(set(term_dict))

In [ ]:
#定義func來計算tf (use numpy to create array)。
def termFrequency(doctoken):
    tf_vector = np.zeros([len(term_dict),1])
    for t in doctoken:
        tf_vector[term_dict.index(t),0] = doctoken.count(t)
    return tf_vector

tf_doc = []
for d in range(0,len(content_list)):
    tf_doc.append(termFrequency(content_list[d]))

In [68]:
#用numpy產生空的idf_vector，定義func計算idf (先求df再利用判斷式判斷df值，並藉此計算idf值，後塞入idf_vector)。
idf_vector = np.zeros([len(term_dict),1])
def inverseDocumentFrequency(term):
    numDocumentsWithThisTerm = 0
    for doc in range(0,len(content_list)):
            if term in content_list[doc]:
                numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
    
    if numDocumentsWithThisTerm > 0:
        idf_vector[term_dict.index(term),0] = math.log10(float(len(content_list)) / numDocumentsWithThisTerm)
    else:
        idf_vector[term_dict.index(term),0] = 0
    return idf_vector

for t in term_dict:
    inverseDocumentFrequency(t)

In [69]:
#求tfidf，迴圈對每篇document的tf值去乘上idf值。
tfidf_doc = []
for doc in range(0,len(tf_doc)):
    tfidf_doc.append(tf_doc[doc]*idf_vector)

In [70]:
#定義func，一個計算tfidf值的平方和開根號，另一個計算相似度。
def sqrt_sum_square(tfidf):
    return np.sqrt(sum(np.square(tfidf)))
    
def similarity(tfidf1,tfidf2): 
    return (np.dot(tfidf1.T,tfidf2)[0][0]) / ((sqrt_sum_square(tfidf1)[0]) * (sqrt_sum_square(tfidf2)[0]))

In [72]:
#for迴圈計算各doc與初、中、中高級字典的相似度（因為起始值為0所以為7332,7333,7334）。
sim1 = []
for i in range(0,len(tfidf_doc)):
    sim1.append(similarity(tfidf_doc[7332],tfidf_doc[i]))

sim2 = []
for j in range(0,len(tfidf_doc)):
    sim2.append(similarity(tfidf_doc[7333],tfidf_doc[j]))

sim3 = []
for k in range(0,len(tfidf_doc)):
    sim3.append(similarity(tfidf_doc[7334],tfidf_doc[k]))

In [150]:
#運用邏輯判斷式，判斷各篇文章的類別（初級：1,中級：2,中高級：3）
Similar_Label = []
for i in range(0,len(sim1)):
    if sim1[i] > sim2[i] and sim1[i] > sim3[i]:
        Similar_Label.append('1')
    elif sim2[i] > sim1[i] and sim2[i] > sim3[i]:
        Similar_Label.append('2')
    else:
        Similar_Label.append('3')

In [244]:
#使用pandas dataframe 儲存文章內文，類別，相似度分類
d = {'Content' : pd.Series(news),
     'Catagory' : pd.Series(catagory),
     'Similar_Label' : pd.Series(Similar_Label[0:7332])}
df = pd.DataFrame(d)

In [245]:
#存成csv檔匯出
df.to_csv('cosine_similarity.csv',header=True)

In [246]:
#另存一個包含3個相似度的，免得之後要重跑一次
d = {'Content' : pd.Series(news),
     'Catagory' : pd.Series(catagory),
     'Similar_Label' : pd.Series(Similar_Label[0:7332]),
     'Gept1_Similar' : pd.Series(sim1[0:7332]),
     'Gept2_Similar' : pd.Series(sim2[0:7332]),
     'Gept3_Similar' : pd.Series(sim3[0:7332])}
df = pd.DataFrame(d)

In [248]:
#存成csv檔匯出
df.to_csv('cosine_similarity_with_sim1to3.csv',header=True)

In [249]:
#我用這個驗證位置有沒有出錯
for i in range(0,7333):
    if Similar_Label[i] == '3':
        print(i)

310
496
737
1044
1262
1409
1627
1833
2051
2313
2531
2859
3077
3466
3684
4136
4354
4873
5091
5675
5893
6541
6759
